In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the processed data
df = pd.read_parquet("../data/dp1_processed_full.pq")

# Get spectroscopic and photometric redshifts
z_cat = df.query("zspec_survey != '-'")
zspec = z_cat.z_best
zphot = np.median(z_cat[["z_LePhare", "z_EAzY", "z_z-phot"]], axis=1)

In [ ]:
fig, axes = plt.subplot_mosaic(
    """
    zzpp
    zzss
    """,
    figsize=(7, 3.0),
    dpi=150,
)


u = (2.25, 3.25)
g = (3.25, 4.25)
r = (4.25, 5.25)

# Plot photo-z vs spec-z
# ----------------------
axes["z"].scatter(zspec, zphot, s=1, marker=".")

settings = dict(lw=0, alpha=0.25, zorder=-1)
axes["z"].fill_between(u, 0, u[1], color="C4", **settings)
axes["z"].fill_between([0, u[0]], u[0], u[1], color="C4", **settings)

axes["z"].fill_between(g, 0, g[1], color="C2", **settings)
axes["z"].fill_between([0, g[0]], g[0], g[1], color="C2", **settings)

axes["z"].fill_between(r, 0, r[1], color="C3", **settings)
axes["z"].fill_between([0, r[0]], r[0], r[1], color="C3", **settings)

axes["z"].set(
    xlim=(0, 5.1), ylim=(0, 5.1), xlabel="Spec-z", ylabel="Photo-z", aspect="equal"
)

# Plot histograms of photo-z and spec-z selections
# ------------------------------------------------
settings = dict(lw=1, histtype="step", bins=20, range=(0, 5.1), density=True)

axes["p"].hist(zphot[(zspec > u[0]) & (zspec < u[1])], color="C4", **settings)
axes["p"].hist(zphot[(zspec > g[0]) & (zspec < g[1])], color="C2", **settings)
axes["p"].hist(zphot[(zspec > r[0]) & (zspec < r[1])], color="C3", **settings)
axes["p"].set(xlabel="Photo-z", ylabel="Density", xlim=(-0.1, 5.2))
axes["p"].text(0.1, 1.6, "Spec-z selection")

axes["s"].hist(zspec[(zphot > u[0]) & (zphot < u[1])], color="C4", **settings)
axes["s"].hist(zspec[(zphot > g[0]) & (zphot < g[1])], color="C2", **settings)
axes["s"].hist(zspec[(zphot > r[0]) & (zphot < r[1])], color="C3", **settings)
axes["s"].set(xlabel="Spec-z", ylabel="Density", xlim=(-0.1, 5.2))
axes["s"].text(0.1, 1.4, "Photo-z selection")

fig.subplots_adjust(wspace=0.9, hspace=0.65)

fig.savefig("../figures/astrodeep_photoz.pdf", bbox_inches="tight")